In [23]:
import requests
#from selenium import webdriver
#from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import time
import re


In [33]:
output_dir = os.path.join("..", "data")

In [172]:
url = "https://api.chess.com/pub/player/Hikaru"
headers = {"User-Agent": "Mozilla/5.0"}
r = requests.get(url, headers=headers)

In [28]:
def get_leaderboard_page(game_type="live", page=1):
    """
    Devuelve la lista de jugadores de una página del leaderboard de Chess.com.
    """
    url = f"https://www.chess.com/callback/leaderboard/{game_type}?gameType={game_type}&page={page}&totalPage=10000"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    r = requests.get(url, headers=headers)
    if r.status_code != 200:
        print(f"Error en página {page}: {r.status_code}")
        return []
    
    data = r.json()
    leaders = data.get("leaders", [])
    
    players = []
    for l in leaders:
        u = l["user"]
        players.append({
            "rank": l.get("rank"),
            "username": u.get("username"),
            "title": u.get("chess_title"),
            "country": u.get("country_name"),
            "rating": l.get("score"),
            "games_played": l.get("totalGameCount"),
            "wins": l.get("totalWinCount"),
            "losses": l.get("totalLossCount"),
            "draws": l.get("totalDrawCount")
        })
    return players

In [29]:
all_players = []
for page in range(1, 6):  # páginas 1 a 5
    print(f"Procesando página {page}...")
    players = get_leaderboard_page("live", page)
    all_players.extend(players)

df = pd.DataFrame(all_players)
df.head(10) 

Procesando página 1...
Procesando página 2...
Procesando página 3...
Procesando página 4...
Procesando página 5...


,rank,username,title,country,rating,games_played,wins,losses,draws
0,1,Hikaru,GM,United States,3400,43196,33798,5291,4107
1,2,MagnusCarlsen,GM,Norway,3367,5591,4040,867,684
2,3,Firouzja2003,GM,France,3268,14303,7959,4568,1776
3,4,LyonBeast,GM,France,3249,3345,2092,746,507
4,5,lachesisQ,GM,Russia,3232,7493,4312,2169,1012
5,6,gurelediz,GM,Türkiye,3226,5369,2697,2141,531
6,7,DenLaz,GM,Belarus,3225,4593,2761,1114,718
7,8,GMWSO,GM,United States,3201,3937,2267,958,712
8,9,spicycaterpillar,GM,United States,3200,5905,3580,1710,615
9,10,0gZPanda,IM,United States,3198,5478,2932,1994,552


In [ ]:
output_path = os.path.join(output_dir, "chess_ratings.csv")
df.to_csv(output_path, index=False, encoding="utf-8")

In [37]:
ECO_DICT = pd.read_csv("../data/eco_dict.csv").set_index("eco")["name"].to_dict()


In [ ]:
def get_month_games(username, year=datetime.today().year, month=datetime.today().month):
    """
    Descarga y limpia todas las partidas de un mes específico para un jugador.
    - Si no se especifica year y month, usa el mes actual.
    """
    url = f"https://api.chess.com/pub/player/{username.lower()}/games/{year}/{month:02d}"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    r = requests.get(url, headers=headers)
    if r.status_code != 200:
        return pd.DataFrame()
    
    games = r.json().get("games", [])
    
    clean = []
    for g in games:
        pgn = g.get("pgn", "")
        
        # extraer código ECO desde el PGN
        match_code = re.search(r'\[ECO "([A-E]\d{2})"\]', pgn)
        eco_code = match_code.group(1) if match_code else None
        eco_name = ECO_DICT.get(eco_code, None)  # buscar nombre desde dict
        
        clean.append({
            "url": g.get("url"),
            "pgn": pgn,
            "time_control": g.get("time_control"),
            "time_class": g.get("time_class"),
            "accuracies_white": g.get("accuracies", {}).get("white"),
            "accuracies_black": g.get("accuracies", {}).get("black"),
            "white_username": g["white"]["username"],
            "white_rating": g["white"]["rating"],
            "white_result": g["white"]["result"],
            "black_username": g["black"]["username"],
            "black_rating": g["black"]["rating"],
            "black_result": g["black"]["result"],
            "eco": eco_code,
            "eco_name": eco_name,   
            "rules": g.get("rules"),
        })
    
    return pd.DataFrame(clean)

In [40]:
def get_last_3_months_games(usernames):
    """
    Devuelve un DataFrame con las partidas de los últimos 3 meses
    para todos los jugadores en la lista `usernames`.
    """
    all_games = []
    
    # Año y mes actual
    today = datetime.today()
    year, month = today.year, today.month
    
    # Calcular últimos 3 meses (corrigiendo cambio de año)
    months = [(year, month - i) for i in range(3)]
    months = [(y if m > 0 else y-1, (m-1)%12+1) for (y,m) in months]

    i = 1
    for user in usernames:
        print(f"Descargando partidas de {user}...")
        print(i)
        for y, m in months:
            df_games = get_month_games(user, y, m)  # usamos tu función limpia
            if not df_games.empty:
                all_games.append(df_games)
        time.sleep(1)  # para no saturar la API
        i +=1

    if all_games:
        return pd.concat(all_games, ignore_index=True)
    else:
        return pd.DataFrame()
    

In [41]:
def map_result(color, result):
    if result == "win":
        return 1 if color == "white" else 0
    elif result in {"resigned", "checkmated", "timeout", "abandoned"}:
        return 0 if color == "white" else 1
    elif result in {"repetition", "insufficient", "agreed", "timevsinsufficient", "stalemate", "50move"}:
        return 0.5

In [42]:
usernames = df["username"].tolist()
games_df = get_last_3_months_games(usernames)

Descargando partidas de Hikaru...
1
Descargando partidas de MagnusCarlsen...
2
Descargando partidas de Firouzja2003...
3
Descargando partidas de LyonBeast...
4
Descargando partidas de lachesisQ...
5
Descargando partidas de gurelediz...
6
Descargando partidas de DenLaz...
7
Descargando partidas de GMWSO...
8
Descargando partidas de spicycaterpillar...
9
Descargando partidas de 0gZPanda...
10
Descargando partidas de FairChess_on_YouTube...
11
Descargando partidas de rpragchess...
12
Descargando partidas de NikoTheodorou...
13
Descargando partidas de Philippians46...
14
Descargando partidas de GHANDEEVAM2003...
15
Descargando partidas de FaustinoOro...
16
Descargando partidas de Javokhir_Sindarov05...
17
Descargando partidas de TRadjabov...
18
Descargando partidas de jefferyx...
19
Descargando partidas de artooon...
20
Descargando partidas de Baku_Boulevard...
21
Descargando partidas de Sina-Movahed...
22
Descargando partidas de Parhamov...
23
Descargando partidas de Arseniy_Nesterov...
2

C:\Users\enriq\AppData\Local\Temp\ipykernel_5840\1359876324.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_games, ignore_index=True)


In [43]:
games = games_df.drop_duplicates()
games = games[games["time_class"] == "blitz"]
games = games[games["rules"] == "chess"]
games = games.reset_index(drop=True)
games["score_white"] = games["white_result"].apply(lambda r: map_result("white", r))
games["score_black"] = games["black_result"].apply(lambda r: map_result("black", r))
output_path = os.path.join(output_dir, "chess_games.csv")
games.to_csv(output_path, index=False, encoding="utf-8")

In [44]:
games.head()

,url,pgn,time_control,time_class,accuracies_white,accuracies_black,white_username,white_rating,white_result,black_username,black_rating,black_result,eco,eco_name,rules,score_white,score_black
0,https://www.chess.com/game/live/151138695137,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,86.76,92.54,yesilikeorangejuice,2645,resigned,Hikaru,3400,win,B06,Modern Defense,chess,0.0,0.0
1,https://www.chess.com/game/live/151139296381,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,94.14,87.00,Hikaru,3401,win,Aygehovit1992,2820,resigned,A48,East Indian Defense,chess,1.0,1.0
2,https://www.chess.com/game/live/151140489263,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,88.51,88.33,RobertoJBM,2948,repetition,Hikaru,3394,repetition,A04,Colle System: Rhamphorhynchus Variation,chess,0.5,0.5
3,https://www.chess.com/game/live/151141090603,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,92.03,80.57,Artemiki,2720,win,Hikaru,3378,resigned,B06,Modern Defense,chess,1.0,1.0
4,https://www.chess.com/game/live/151141697249,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,95.00,86.54,Hikaru,3378,win,soul_mortall,2555,resigned,A01,Nimzo-Larsen Attack,chess,1.0,1.0


In [146]:
players = pd.concat([
    games[["white_username", "white_rating", "score_white"]]
        .rename(columns={"white_username": "player",
                         "white_rating": "rating",
                         "score_white": "score"}),
    games[["black_username", "black_rating", "score_black"]]
        .rename(columns={"black_username": "player",
                         "black_rating": "rating",
                         "score_black": "score"})
])

stats = players.groupby("player").agg(
    games_played=("score", "count"),
    avg_score=("score", "mean"),
    wins=("score", lambda x: (x==1).sum()),
    draws=("score", lambda x: (x==0.5).sum()),
    losses=("score", lambda x: (x==0).sum()),
    avg_rating=("rating", "mean")
).sort_values("games_played", ascending=False)

stats = stats[(stats["games_played"] > 20) & (stats.index.isin(usernames))]


In [150]:
stats

,games_played,avg_score,wins,draws,losses,avg_rating
player,,,,,,
DanielNaroditsky,2819,0.520752,1328,280,1211,3069.789287
Twitch_ElhamBlitz05,1836,0.550381,902,217,717,3042.900871
bancsoo,1810,0.522099,838,214,758,3049.549171
ChessMaster2011_1,1411,0.521616,654,164,593,2871.537916
Yaacovn,1325,0.524151,619,151,555,2948.098113
...,...,...,...,...,...,...
ChessFighter_2011,25,0.520000,13,0,12,3045.120000
KNVB,23,0.695652,16,0,7,3023.000000
GukeshDommaraju,22,0.590909,13,0,9,3049.409091


In [162]:
df.head()

,rank,username,title,country,rating,games_played,wins,losses,draws
0,1,Hikaru,GM,United States,3395,43183,33786,5291,4106
1,2,MagnusCarlsen,GM,Norway,3367,5591,4040,867,684
2,3,Firouzja2003,GM,France,3268,14303,7959,4568,1776
3,4,LyonBeast,GM,France,3249,3345,2092,746,507
4,5,lachesisQ,GM,Russia,3232,7493,4312,2169,1012


In [165]:
def build_players_table(games, leaderboard):
    players = pd.concat([
        games[["white_username", "white_rating", "score_white"]]
            .rename(columns={"white_username": "player",
                             "white_rating": "rating",
                             "score_white": "score"}),
        games[["black_username", "black_rating", "score_black"]]
            .rename(columns={"black_username": "player",
                             "black_rating": "rating",
                             "score_black": "score"})
    ])

    stats = players.groupby("player").agg(
        games_played=("score", "count"),
        avg_score=("score", "mean"),
        wins=("score", lambda x: (x == 1).sum()),
        draws=("score", lambda x: (x == 0.5).sum()),
        losses=("score", lambda x: (x == 0).sum()),
        avg_rating=("rating", "mean")
    ).reset_index()

    leaderboard_players = leaderboard[["username", "rank", "title", "country"]]

    players_table = stats.merge(
        leaderboard_players,
        left_on="player", right_on="username", how="inner"
    ).drop(columns="username")

    return players_table

In [167]:
jugadores = build_players_table(games,df)

In [171]:
jugadores[jugadores["title"].isna()]

,player,games_played,avg_score,wins,draws,losses,avg_rating,rank,title,country
115,Mighty_Gladius16,51,0.509804,22,8,21,2946.607843,220,None,India
184,chesterreyes,2,0.500000,1,0,1,2978.500000,229,None,Philippines
222,ricky8632200,13,0.269231,2,3,8,3014.461538,125,None,China


In [175]:
import pandas as pd
import os

output_dir = "../data"

games = pd.read_csv(os.path.join(output_dir, "games.csv"))
leaders = pd.read_csv(os.path.join(output_dir, "chess_ratings.csv"))

print(games.shape, leaders.shape)
games.head()

(53668, 16) (250, 9)


,url,pgn,time_control,time_class,accuracies_white,accuracies_black,white_username,white_rating,white_result,black_username,black_rating,black_result,eco,rules,score_white,score_black
0,https://www.chess.com/game/live/151138695137,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,86.76,92.54,yesilikeorangejuice,2645,resigned,Hikaru,3400,win,https://www.chess.com/openings/Modern-Defense-...,chess,0.0,0.0
1,https://www.chess.com/game/live/151139296381,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,94.09,86.99,Hikaru,3401,win,Aygehovit1992,2820,resigned,https://www.chess.com/openings/Indian-Game-Kni...,chess,1.0,1.0
2,https://www.chess.com/game/live/151140489263,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,88.51,88.33,RobertoJBM,2948,repetition,Hikaru,3394,repetition,https://www.chess.com/openings/Reti-Opening-Du...,chess,0.5,0.5
3,https://www.chess.com/game/live/151141090603,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,92.03,80.57,Artemiki,2720,win,Hikaru,3378,resigned,https://www.chess.com/openings/Modern-Defense-...,chess,1.0,1.0
4,https://www.chess.com/game/live/151141697249,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,95.00,86.54,Hikaru,3378,win,soul_mortall,2555,resigned,https://www.chess.com/openings/Nimzowitsch-Lar...,chess,1.0,1.0


In [190]:
games["eco"][0].split("/")[-1]

'Modern-Defense-with-1-e4...3.Nf3-c6-4.Bd3-d6'

In [197]:
games["eco"].isna().sum()


0

In [34]:
def get_player_info(username: str):
    """
    Devuelve nombre y avatar de un jugador de Chess.com.
    """
    url = f"https://api.chess.com/pub/player/{username}"
    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(url, headers=headers)
    time.sleep(0.5)  # pausa para no saturar la API
    
    if r.status_code == 200:
        data = r.json()
        return {
            "name": data.get("name"),
            "avatar": data.get("avatar")
        }
    return {"name": None, "avatar": None}


In [ ]:
def add_flag_url(df):
    df["country_code"] = df["country"].str.split("/").str[-1].str.lower()
    df["flag_url"] = "https://flagcdn.com/h20/" + df["country_code"] + ".png"
    return df

In [44]:
url = f"https://api.chess.com/pub/player/xamax2000"
headers = {"User-Agent": "Mozilla/5.0"}
r = requests.get(url, headers=headers)
r.json()

{'player_id': 26608680,
 '@id': 'https://api.chess.com/pub/player/xamax2000',
 'url': 'https://www.chess.com/member/xamax2000',
 'name': 'Max Warmerdam',
 'username': 'xamax2000',
 'title': 'GM',
 'followers': 312,
 'country': 'https://api.chess.com/pub/country/NL',
 'location': 'Tegelen',
 'last_online': 1754734203,
 'joined': 1455002600,
 'status': 'premium',
 'is_streamer': False,
 'verified': False,
 'league': 'Legend',
 'streaming_platforms': []}

In [16]:
leaders = pd.read_csv(os.path.join(output_dir, "chess_ratings.csv"))
leaders

,rank,username,title,country,rating,games_played,wins,losses,draws
0,1,Hikaru,GM,United States,3395,43183,33786,5291,4106
1,2,MagnusCarlsen,GM,Norway,3367,5591,4040,867,684
2,3,Firouzja2003,GM,France,3268,14303,7959,4568,1776
3,4,LyonBeast,GM,France,3249,3345,2092,746,507
4,5,lachesisQ,GM,Russia,3232,7493,4312,2169,1012
...,...,...,...,...,...,...,...,...,...
245,246,qsjtgx,IM,China,2962,8125,3812,3593,720
246,247,platy3,GM,Spain,2962,4245,2376,1337,532
247,248,SPEEDSKATER,GM,United States,2962,8107,4490,2670,947
248,249,AhmadzadaA,GM,Azerbaijan,2961,2055,979,781,295


In [19]:
players = pd.read_csv(os.path.join(output_dir, "players_stats.csv"))


In [35]:
# --- construir stats recientes desde games ---
players = pd.concat([
    games[["white_username", "white_rating", "score_white"]]
        .rename(columns={"white_username": "player",
                         "white_rating": "rating",
                         "score_white": "score"}),
    games[["black_username", "black_rating", "score_black"]]
        .rename(columns={"black_username": "player",
                         "black_rating": "rating",
                         "score_black": "score"})
])



In [36]:
stats = players.groupby("player").agg(
    games_played=("score", "count"),
    avg_score=("score", "mean"),
    wins=("score", lambda x: (x == 1).sum()),
    draws=("score", lambda x: (x == 0.5).sum()),
    losses=("score", lambda x: (x == 0).sum())
).reset_index()


In [37]:
# --- merge con leaderboard (stats globales) ---
players_table = stats.merge(
    leaders[["username", "rank", "title", "country", "rating",
             "games_played", "wins", "losses", "draws"]],
    left_on="player", right_on="username", how="inner"
).drop(columns="username")


In [40]:
# --- renombrar columnas ---
players_table = players_table.rename(columns={
    "games_played_x": "games_played_recent",
    "wins_x": "wins_recent",
    "draws_x": "draws_recent",
    "losses_x": "losses_recent",
    "games_played_y": "games_played_total",
    "wins_y": "wins_total",
    "draws_y": "draws_total",
    "losses_y": "losses_total"
    })


In [41]:
players_table.head(2)

,player,games_played_recent,avg_score,wins_recent,draws_recent,losses_recent,rank,title,country,rating,games_played_total,wins_total,losses_total,draws_total
0,0gZPanda,120,0.620833,66,17,37,12,IM,United States,3179,5471,2926,1993,552
1,A-Adly,85,0.635294,50,8,27,128,GM,Egypt,3026,1151,655,401,95


In [42]:
# --- añadir nombre y avatar ---
players_table[["name", "avatar"]] = players_table["player"].apply(
    lambda u: pd.Series(get_player_info(u))
)

# --- ordenar columnas ---
players_table = players_table[
    ["player", "name", "avatar", "rank", "title", "country", "rating",
     "games_played_recent", "wins_recent", "draws_recent", "losses_recent",
     "games_played_total", "wins_total", "draws_total", "losses_total",
     "avg_score"]
]

In [71]:
players_table = add_flag_url(players_table)

In [66]:
output_path = os.path.join(output_dir, "players_stats.csv")
players_table.to_csv(output_path, index=False, encoding="utf-8")

In [ ]:
url = "https://api.chess.com/pub/tournament/titled-tuesday-blitz-september-09-2025-5905665"
r = requests.get(url, headers=headers)
t = r.json()


In [87]:
rounds = requests.get("https://api.chess.com/pub/tournament/titled-tuesday-blitz-september-09-2025-5905665/11/1",headers = headers)

In [86]:
rounds.json()

{'code': 0,
 'message': 'Tournament "titled-tuesday-blitz-september-09-2025-5905665" doesn\'t have group "1" in round "10".'}

In [63]:
players_table = pd.read_csv(os.path.join(output_dir, "players_stats.csv"))

In [90]:
DATA_DIR = "../data"
games = pd.read_csv(os.path.join(DATA_DIR, "chess_games.csv"))


In [92]:
games

,url,pgn,time_control,time_class,accuracies_white,accuracies_black,white_username,white_rating,white_result,black_username,black_rating,black_result,eco,eco_name,rules,score_white,score_black
0,https://www.chess.com/game/live/151138695137,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,86.76,92.54,yesilikeorangejuice,2645,resigned,Hikaru,3400,win,B06,Modern Defense,chess,0.0,0.0
1,https://www.chess.com/game/live/151139296381,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,94.14,87.00,Hikaru,3401,win,Aygehovit1992,2820,resigned,A48,East Indian Defense,chess,1.0,1.0
2,https://www.chess.com/game/live/151140489263,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,88.51,88.33,RobertoJBM,2948,repetition,Hikaru,3394,repetition,A04,Colle System: Rhamphorhynchus Variation,chess,0.5,0.5
3,https://www.chess.com/game/live/151141090603,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,92.03,80.57,Artemiki,2720,win,Hikaru,3378,resigned,B06,Modern Defense,chess,1.0,1.0
4,https://www.chess.com/game/live/151141697249,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300,blitz,95.00,86.54,Hikaru,3378,win,soul_mortall,2555,resigned,A01,Nimzo-Larsen Attack,chess,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53305,https://www.chess.com/game/live/141789898502,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,blitz,89.97,84.27,SuperPower2011,2949,win,ScacchiParlanoDaSoli,2722,checkmated,E91,King's Indian Defense: Kazakh Variation,chess,1.0,1.0
53306,https://www.chess.com/game/live/141790118426,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,blitz,75.08,84.34,ScacchiParlanoDaSoli,2719,resigned,SuperPower2011,2952,win,B15,Caro-Kann Defense,chess,0.0,0.0
53307,https://www.chess.com/game/live/141790279374,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,blitz,91.04,84.69,SuperPower2011,2955,win,ScacchiParlanoDaSoli,2716,resigned,B76,"Sicilian Defense: Dragon Variation, Yugoslav A...",chess,1.0,1.0
53308,https://www.chess.com/game/live/141790450442,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,blitz,80.10,85.62,ScacchiParlanoDaSoli,2713,timeout,SuperPower2011,2958,win,B50,Sicilian Defense,chess,0.0,0.0


In [15]:
url = f"https://api.chess.com/pub/player/Hikaru/games/2025/09"
headers = {"User-Agent": "Mozilla/5.0"}
    
r = requests.get(url, headers=headers)

In [45]:
url = "https://api.chess.com/pub/player/Hikaru/tournaments"
headers = {"User-Agent": "Mozilla/5.0"}
    
r = requests.get(url, headers=headers)

In [57]:
players

[{'rank': 201,
  'username': 'Marcan2b',
  'title': 'GM',
  'country': 'France',
  'rating': 2996,
  'games_played': 12222,
  'wins': 6475,
  'losses': 4924,
  'draws': 823},
 {'rank': 202,
  'username': 'Shankland',
  'title': 'GM',
  'country': 'United States',
  'rating': 2996,
  'games_played': 4736,
  'wins': 2547,
  'losses': 1494,
  'draws': 695},
 {'rank': 203,
  'username': 'DrVelja',
  'title': 'GM',
  'country': 'Serbia',
  'rating': 2994,
  'games_played': 5206,
  'wins': 2833,
  'losses': 1912,
  'draws': 461},
 {'rank': 204,
  'username': 'Anton_Demchenko',
  'title': 'GM',
  'country': 'Slovenia',
  'rating': 2994,
  'games_played': 15035,
  'wins': 7720,
  'losses': 5596,
  'draws': 1719},
 {'rank': 205,
  'username': 'monomita',
  'title': 'IM',
  'country': 'India',
  'rating': 2993,
  'games_played': 2325,
  'wins': 1105,
  'losses': 939,
  'draws': 281},
 {'rank': 206,
  'username': 'tptagain',
  'title': 'GM',
  'country': 'Spain',
  'rating': 2991,
  'games_played